In [71]:
import re
import os

import pandas as pd
import pandas_read_xml as pdx

from xml.etree import ElementTree as ET


import warnings
warnings.filterwarnings('ignore')

In [40]:
file = './data/sec-edgar-filings/IBM/4/0001562180-21-002509/filing-details.xml'
tree = ET.parse(file)
root = tree.getroot()

In [41]:
issuerName = root.find('issuer/issuerName')
issuerName.text

'INTERNATIONAL BUSINESS MACHINES CORP'

In [42]:
issuerTradingSymbol = root.find('issuer/issuerTradingSymbol')
issuerTradingSymbol.text

'IBM'

In [43]:
rptOwnerName = root.find('reportingOwner/reportingOwnerId/rptOwnerName')
rptOwnerName.text

'KAVANAUGH JAMES J'

In [44]:
officerTitle = root.find('reportingOwner/reportingOwnerRelationship/officerTitle')
officerTitle.text

'Sr. VP and CFO'

In [63]:
transactionDate = ""
transactionAmounts = ""
transactionType = ""
sharesOwnedFollowingTransaction = ""
transactionPricePerShare = ""
if len(list(root.find('nonDerivativeTable'))) != 0:
    transactionDate = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionDate/value')
    transactionAmounts = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionShares/value')
    transactionAcquiredDisposedCode = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionAcquiredDisposedCode/value')
    sharesOwnedFollowingTransaction = root.find('nonDerivativeTable/nonDerivativeTransaction/postTransactionAmounts/sharesOwnedFollowingTransaction/value')
    transactionPricePerShare = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionPricePerShare/value')
    transactionType = transactionAcquiredDisposedCode.text
elif len(list(root.find('derivativeTable'))) != 0:
    transactionDate = root.find('derivativeTable/derivativeTransaction/transactionDate/value')
    transactionAmounts = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionShares/value')
    transactionAcquiredDisposedCode = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionAcquiredDisposedCode/value')
    sharesOwnedFollowingTransaction = root.find('derivativeTable/derivativeTransaction/postTransactionAmounts/sharesOwnedFollowingTransaction/value')
    transactionPricePerShare = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionPricePerShare/value')
    transactionType = transactionAcquiredDisposedCode.text
if transactionType == 'A':
    transactionType = "Buy"
elif transactionType == "D":
    transactionType = "Sell"
transactionPricePerShare.text

'133.26'

In [53]:
len(list(root.find('nonDerivativeTable')))

0

In [37]:
dtransactionDate = root.find('derivativeTable/derivativeTransaction/transactionDate/value')
dtransactionDate.text

AttributeError: 'NoneType' object has no attribute 'text'

In [9]:
transactionAmounts = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionShares/value')
transactionAmounts.text

'23.00'

In [10]:
transactionType = ""
transactionAcquiredDisposedCode = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionAcquiredDisposedCode/value')
if transactionAcquiredDisposedCode.text == 'A':
    transactionType = "Buy"
elif transactionAcquiredDisposedCode.text == "D":
    transactionType = "Sell"
transactionType

'Buy'

In [11]:
sharesOwnedFollowingTransaction = root.find('derivativeTable/derivativeTransaction/postTransactionAmounts/sharesOwnedFollowingTransaction/value')
sharesOwnedFollowingTransaction.text

'7886.00'

In [12]:
transactionPricePerShare = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionPricePerShare/value')
transactionPricePerShare.text

'133.26'

In [13]:
form4_df = pd.DataFrame(columns=['Date', 'Ticker', 'Company Name', 'Reporter', 'Title', 'Type', 'Shares', 'Price', 'Total Shares'])

In [14]:
new_form4 =pd.Series( [transactionDate.text, issuerTradingSymbol.text, issuerName.text, rptOwnerName.text, officerTitle.text, transactionType, transactionAmounts.text, transactionPricePerShare.text, sharesOwnedFollowingTransaction.text] , index=form4_df.columns )

In [15]:
form4_df = form4_df.append( new_form4, ignore_index=True)
form4_df

,Date,Ticker,Company Name,Reporter,Title,Type,Shares,Price,Total Shares
0,2021-03-31,IBM,INTERNATIONAL BUSINESS MACHINES CORP,KAVANAUGH JAMES J,Sr. VP and CFO,Buy,23.00,133.26,7886.00


In [69]:
form4_df = pd.DataFrame(columns=['Date', 'Ticker', 'Company Name', 'Reporter', 'Title', 'Type', 'Shares', 'Price', 'Total Shares'])

def parseForm4(file, df):
    tree = ET.parse(file)
    root = tree.getroot()
    issuerName = root.find('issuer/issuerName')
    issuerTradingSymbol = root.find('issuer/issuerTradingSymbol')
    rptOwnerName = root.find('reportingOwner/reportingOwnerId/rptOwnerName')
    officerTitle = root.find('reportingOwner/reportingOwnerRelationship/officerTitle')
    
    
    transactionDate = ""
    transactionAmounts = ""
    transactionType = ""
    sharesOwnedFollowingTransaction = ""
    transactionPricePerShare = ""
    if len(list(root.find('nonDerivativeTable'))) != 0:
        transactionDate = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionDate/value')
        transactionAmounts = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionShares/value')
        transactionAcquiredDisposedCode = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionAcquiredDisposedCode/value')
        sharesOwnedFollowingTransaction = root.find('nonDerivativeTable/nonDerivativeTransaction/postTransactionAmounts/sharesOwnedFollowingTransaction/value')
        transactionPricePerShare = root.find('nonDerivativeTable/nonDerivativeTransaction/transactionAmounts/transactionPricePerShare/value')
        transactionType = transactionAcquiredDisposedCode.text
    elif len(list(root.find('derivativeTable'))) != 0:
        transactionDate = root.find('derivativeTable/derivativeTransaction/transactionDate/value')
        transactionAmounts = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionShares/value')
        transactionAcquiredDisposedCode = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionAcquiredDisposedCode/value')
        sharesOwnedFollowingTransaction = root.find('derivativeTable/derivativeTransaction/postTransactionAmounts/sharesOwnedFollowingTransaction/value')
        transactionPricePerShare = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionPricePerShare/value')
        transactionType = transactionAcquiredDisposedCode.text
    if transactionType == 'A':
        transactionType = "Buy"
    elif transactionType == "D":
        transactionType = "Sell"
    else:
        transactionType = "Other"
    sharesOwnedFollowingTransaction = root.find('derivativeTable/derivativeTransaction/postTransactionAmounts/sharesOwnedFollowingTransaction/value')
    transactionPricePerShare = root.find('derivativeTable/derivativeTransaction/transactionAmounts/transactionPricePerShare/value')
    new_form4 =pd.Series( [transactionDate.text, issuerTradingSymbol.text, issuerName.text, rptOwnerName.text, officerTitle.text, transactionType, transactionAmounts.text, transactionPricePerShare.text, sharesOwnedFollowingTransaction.text] , index=form4_df.columns )
    df = df.append( new_form4, ignore_index=True)
    return df
    
form4_df = parseForm4('./data/sec-edgar-filings/IBM/4/0001562180-21-002509/filing-details.xml', form4_df)
form4_df = parseForm4('./data/sec-edgar-filings/IBM/4/0001562180-21-002513/filing-details.xml', form4_df)
form4_df = parseForm4('./data/sec-edgar-filings/IBM/4/0001562180-21-002518/filing-details.xml', form4_df)
form4_df = parseForm4('./data/sec-edgar-filings/IBM/4/0001562180-21-002521/filing-details.xml', form4_df)
form4_df = parseForm4('./data/sec-edgar-filings/MMM/4/0001127602-21-004276/filing-details.xml', form4_df)
form4_df = parseForm4('./data/sec-edgar-filings/MMM/4/0001127602-21-004278/filing-details.xml', form4_df)
# form4_df = parseForm4('./data/sec-edgar-filings/MMM/4/0001127602-21-004629/filing-details.xml', form4_df)
# form4_df = parseForm4('./data/sec-edgar-filings/MMM/4/0001127602-21-006407/filing-details.xml', form4_df)
form4_df = form4_df.set_index(["Ticker", "Date"]) 
form4_df

Company Name            Reporter  \
Ticker Date                                                                   
IBM    2021-03-31  INTERNATIONAL BUSINESS MACHINES CORP   KAVANAUGH JAMES J   
       2021-03-31  INTERNATIONAL BUSINESS MACHINES CORP       TAUREL SIDNEY   
       2021-03-31  INTERNATIONAL BUSINESS MACHINES CORP   HOWARD MICHELLE J   
       2021-03-31  INTERNATIONAL BUSINESS MACHINES CORP     ESKEW MICHAEL L   
MMM    2021-02-04                                 3M CO    Lakkundi Veena M   
       2021-02-05                                 3M CO  Reinseth Theresa E   

                                   Title Type  Shares   Price Total Shares  
Ticker Date                                                                 
IBM    2021-03-31         Sr. VP and CFO  Buy   23.00  133.26      7886.00  
       2021-03-31                   None  Buy  610.00  133.26     44931.00  
       2021-03-31                   None  Buy  549.00  133.26      4972.00  
       2021-03-31                   None  Buy  742.00  133.26     42427.00  
MMM    2021-02-04  Senior Vice President  Buy     178       0          357  
       2021-02-05     Vice President/CAO  Buy     289       0            0

In [80]:
def parse_form4_tickers(tickers, df):
    
    for ticker in tickers:
        directory_in_str = "./data/sec-edgar-filings/"+ticker+"/4/"
        directory = os.fsencode(directory_in_str)

        for root, dirs, files in os.walk(directory, topdown=False):
            for name in files:
                if os.path.splitext(name)[-1] == b'.xml':
                    file_name_path = os.path.join(root, name)
                    ticker_df = parseForm4(os.path.join(root, name), df)

            
df = parse_form4_tickers(['MMM', 'GOOG', 'GOOGL', 'IBM', 'MSFT'])

b'./data/sec-edgar-filings/MMM/4/0001127602-09-005293/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-12-004671/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-11-005097/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-06-000131/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-07-000026/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-05-000008/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-09-026026/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-11-015224/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-16-039523/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-07-000025/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-11-004810/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-18-004641/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-10-014018/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/00011

b'./data/sec-edgar-filings/MMM/4/0001127602-14-016688/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-06-000105/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-13-005420/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-16-039236/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-16-042331/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-14-030505/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-08-009134/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-10-031215/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-18-004588/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-18-032968/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-05-000212/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-03-000050/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-07-000137/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/00011

b'./data/sec-edgar-filings/MMM/4/0000066740-05-000117/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-13-030523/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-15-031100/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-14-004804/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-05-000193/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-15-017216/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-16-067701/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-18-004632/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-15-004289/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-15-004284/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-05-000118/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-18-004655/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-09-011393/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/00011

b'./data/sec-edgar-filings/MMM/4/0001127602-17-025813/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-08-003081/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-06-000156/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-04-000141/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-07-000016/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-20-004082/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-08-009360/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-07-000101/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-06-000004/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-11-012277/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0000066740-05-000167/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-14-015749/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/0001127602-09-017196/filing-details.xml'
b'./data/sec-edgar-filings/MMM/4/00011

b'./data/sec-edgar-filings/GOOG/4/0001209191-19-054959/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001209191-19-026307/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001209191-20-016807/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001209191-17-006898/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001127602-16-041536/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001127602-16-045600/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001209191-17-044449/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001127602-15-032751/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001127602-15-032877/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001209191-19-051335/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001209191-17-051969/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001127602-16-044768/filing-details.xml'
b'./data/sec-edgar-filings/GOOG/4/0001127602-16-050342/filing-details.xml'
b'./data/sec-edgar-filing